# Tratamento de dados

In [1]:
# Importação de bibliotecas
from bs4 import BeautifulSoup
import requests
import pandas as pd

Utilizaremos os dados dos sites 
- [x] [CBF - Série A](https://www.cbf.com.br/futebol-brasileiro/competicoes/campeonato-brasileiro-serie-a)
- [x] [CBF - Série B](https://www.cbf.com.br/futebol-brasileiro/competicoes/campeonato-brasileiro-serie-b)
- [x] [CBF - Série C](https://www.cbf.com.br/futebol-brasileiro/competicoes/campeonato-brasileiro-serie-c)

In [30]:
# URLs para web scraping:
url_serie_a = 'https://www.cbf.com.br/futebol-brasileiro/competicoes/campeonato-brasileiro-serie-a'
url_serie_b = 'https://www.cbf.com.br/futebol-brasileiro/competicoes/campeonato-brasileiro-serie-b'
url_serie_c = 'https://www.cbf.com.br/futebol-brasileiro/competicoes/campeonato-brasileiro-serie-c'

# Fazendo o request para o site:
request_brasileirao_a = requests.get(url_serie_a)
request_brasileirao_b = requests.get(url_serie_b)
request_brasileirao_c = requests.get(url_serie_c)

# Utilizando BeautifulSoup para tratamento HTML:
soup_a = BeautifulSoup(request_brasileirao_a.text, 'html.parser')
soup_b = BeautifulSoup(request_brasileirao_b.text, 'html.parser')
soup_c = BeautifulSoup(request_brasileirao_c.text, 'html.parser')

# Achando as tabelas com as informações:
tabela_a = soup_a.find(class_='table m-b-20 tabela-expandir')
tabela_b = soup_b.find(class_='table m-b-20 tabela-expandir')
tabela_c = soup_c.find(class_='table m-b-20 tabela-expandir')

## Realizando a coleta dos dados:

In [72]:
# Obtendo o título das colunas e a pontuação com <th>
classes_e_pontuacao = []
for i in tabela_a.find_all('th'):
    title = i.text
    classes_e_pontuacao.append(title)

categorias = classes_e_pontuacao[:-20]
pontos = classes_e_pontuacao[20:]

adicionar_tabela = []
# Criando um loop for para preencher os meus dados
for j in tabela_a.find_all('tr')[1:]:
    row_data = j.find_all('td')
    row = [i.text for i in row_data]
    adicionar_tabela.append(row)

# Filtrando os dados da tabela - apenas os das categorias
for info in adicionar_tabela:
    for elemento in info:
        if "Última partida" in elemento:
            adicionar_tabela.remove(info)

# Criando o DataFrame
DadosBrasileirao = pd.DataFrame(columns=categorias)
DadosBrasileirao['PTS'] = pontos

print(DadosBrasileirao)
print(adicionar_tabela)

   Posição PTS    J    V    E    D   GP   GC   SG   CA   CV    % Recentes Próx
0      NaN  54  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN      NaN  NaN
1      NaN  53  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN      NaN  NaN
2      NaN  53  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN      NaN  NaN
3      NaN  50  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN      NaN  NaN
4      NaN  50  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN      NaN  NaN
5      NaN  48  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN      NaN  NaN
6      NaN  48  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN      NaN  NaN
7      NaN  47  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN      NaN  NaN
8      NaN  47  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN      NaN  NaN
9      NaN  46  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN      NaN  NaN
10     NaN  43  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN      NaN  NaN
11     NaN  43  NaN  NaN  NaN  NaN  NaN  NaN  NaN  N